<a href="https://colab.research.google.com/github/rtegao/M5Forecasting/blob/master/FeatureEngineering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importing Libraries

In [0]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow.keras import optimizers, regularizers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout
from sklearn.preprocessing import MinMaxScaler

# Reading Datasets

In [0]:
# Contains information about the dates the products are sold.
df_calendar = pd.read_csv('/content/drive/My Drive/Specialization/Projects/Recurrent Neural Networks/M5 - Data/calendar.csv')

# Contains information about the price of the products sold per store and date.
df_price = pd.read_csv('/content/drive/My Drive/Specialization/Projects/Recurrent Neural Networks/M5 - Data/sell_prices.csv')

#Contains the historical daily unit sales data per product and store.
df_sales = pd.read_csv('/content/drive/My Drive/Specialization/Projects/Recurrent Neural Networks/M5 - Data/sales_train_validation.csv')

In [5]:
df_calendar

,date,wm_yr_wk,weekday,wday,month,year,d,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI
0,2011-01-29,11101,Saturday,1,1,2011,d_1,NaN,NaN,NaN,NaN,0,0,0
1,2011-01-30,11101,Sunday,2,1,2011,d_2,NaN,NaN,NaN,NaN,0,0,0
2,2011-01-31,11101,Monday,3,1,2011,d_3,NaN,NaN,NaN,NaN,0,0,0
3,2011-02-01,11101,Tuesday,4,2,2011,d_4,NaN,NaN,NaN,NaN,1,1,0
4,2011-02-02,11101,Wednesday,5,2,2011,d_5,NaN,NaN,NaN,NaN,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1964,2016-06-15,11620,Wednesday,5,6,2016,d_1965,NaN,NaN,NaN,NaN,0,1,1
1965,2016-06-16,11620,Thursday,6,6,2016,d_1966,NaN,NaN,NaN,NaN,0,0,0
1966,2016-06-17,11620,Friday,7,6,2016,d_1967,NaN,NaN,NaN,NaN,0,0,0
1967,2016-06-18,11621,Saturday,1,6,2016,d_1968,NaN,NaN,NaN,NaN,0,0,0


In [6]:
df_price

,store_id,item_id,wm_yr_wk,sell_price
0,CA_1,HOBBIES_1_001,11325,9.58
1,CA_1,HOBBIES_1_001,11326,9.58
2,CA_1,HOBBIES_1_001,11327,8.26
3,CA_1,HOBBIES_1_001,11328,8.26
4,CA_1,HOBBIES_1_001,11329,8.26
...,...,...,...,...
6841116,WI_3,FOODS_3_827,11617,1.00
6841117,WI_3,FOODS_3_827,11618,1.00
6841118,WI_3,FOODS_3_827,11619,1.00
6841119,WI_3,FOODS_3_827,11620,1.00


In [7]:
df_sales

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,d_5,d_6,d_7,d_8,d_9,d_10,d_11,d_12,d_13,d_14,d_15,d_16,d_17,d_18,d_19,d_20,d_21,d_22,d_23,d_24,d_25,d_26,d_27,d_28,d_29,d_30,d_31,d_32,d_33,d_34,...,d_1874,d_1875,d_1876,d_1877,d_1878,d_1879,d_1880,d_1881,d_1882,d_1883,d_1884,d_1885,d_1886,d_1887,d_1888,d_1889,d_1890,d_1891,d_1892,d_1893,d_1894,d_1895,d_1896,d_1897,d_1898,d_1899,d_1900,d_1901,d_1902,d_1903,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,1,1,3,1,3,1,2,2,0,1,1,1,1,0,0,0,0,0,1,0,4,2,3,0,1,2,0,0,0,1,1,3,0,1,1,1,3,0,1,1
1,HOBBIES_1_002_CA_1_validation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
2,HOBBIES_1_003_CA_1_validation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1,2,2,1,2,1,1,1,0,1,1,1
3,HOBBIES_1_004_CA_1_validation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,1,3,4,2,1,4,1,3,5,0,6,6,0,0,0,0,3,1,2,1,3,1,0,2,5,4,2,0,3,0,1,0,5,4,1,0,1,3,7,2
4,HOBBIES_1_005_CA_1_validation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,1,0,3,2,2,2,3,1,0,0,0,0,1,0,4,4,0,1,4,0,1,0,1,0,1,1,2,0,1,1,2,1,1,0,1,1,2,2,2,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30485,FOODS_3_823_WI_3_validation,FOODS_3_823,FOODS_3,FOODS,WI_3,WI,0,0,2,2,0,3,1,4,1,0,0,3,4,4,0,0,1,0,1,1,7,7,3,6,3,3,7,12,4,2,7,5,12,5,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,2,0,0,0,0,0,1,0,0,1
30486,FOODS_3_824_WI_3_validation,FOODS_3_824,FOODS_3,FOODS,WI_3,WI,0,0,0,0,0,5,0,1,1,3,1,1,0,4,2,0,1,2,1,1,0,0,0,0,3,1,1,1,2,0,1,0,1,1,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,2,0,0,0,0,1,1,1,1,0,0,0,0,0,0,0,0,0,0,1,0
30487,FOODS_3_825_WI_3_validation,FOODS_3_825,FOODS_3,FOODS,WI_3,WI,0,6,0,2,2,4,1,8,5,2,7,5,3,5,20,8,10,3,3,4,7,2,3,5,6,3,4,1,2,5,1,2,2,2,...,1,1,2,0,0,2,0,0,0,2,0,1,1,0,0,0,0,1,1,2,1,4,0,0,2,2,4,0,0,0,2,1,0,2,0,1,0,0,1,0
30488,FOODS_3_826_WI_3_validation,FOODS_3_826,FOODS_3,FOODS,WI_3,WI,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,1,1,1,1,1,1,0,2,1,1,2,4,3,0,0,0,0,0,1,1,2,1,1,1,1,2,0,1,0,3,0,0,1,0,0,1,0,3,1,3


# Filtering a Specific Product

In [0]:
df_food =  df_sales.iloc[df_sales[df_sales['id'] == 'FOODS_3_495_WI_3_validation'].loc[:,'d_1':].index[0],6:]

# Joing DataFrames

In [0]:
df_calendar_price = df_calendar.set_index('wm_yr_wk').join(df_price[(df_price['store_id'] == df_sales[df_sales['id'] == 'FOODS_3_495_WI_3_validation']['store_id'].values[0]) & 
         (df_price['item_id'] == df_sales[df_sales['id'] == 'FOODS_3_495_WI_3_validation']['item_id'].values[0])].set_index('wm_yr_wk')).reset_index()
df_final = df_food.to_frame('qty_sales').reset_index().set_index('index').join(df_calendar_price.set_index('d'))

In [23]:
df_final

,qty_sales,wm_yr_wk,date,weekday,wday,month,year,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,store_id,item_id,sell_price
index,,,,,,,,,,,,,,,,,
d_1,1,11101,2011-01-29,Saturday,1,1,2011,NaN,NaN,NaN,NaN,0,0,0,WI_3,FOODS_3_495,1.77
d_2,2,11101,2011-01-30,Sunday,2,1,2011,NaN,NaN,NaN,NaN,0,0,0,WI_3,FOODS_3_495,1.77
d_3,4,11101,2011-01-31,Monday,3,1,2011,NaN,NaN,NaN,NaN,0,0,0,WI_3,FOODS_3_495,1.77
d_4,2,11101,2011-02-01,Tuesday,4,2,2011,NaN,NaN,NaN,NaN,1,1,0,WI_3,FOODS_3_495,1.77
d_5,3,11101,2011-02-02,Wednesday,5,2,2011,NaN,NaN,NaN,NaN,1,0,1,WI_3,FOODS_3_495,1.77
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
d_1909,1,11612,2016-04-20,Wednesday,5,4,2016,NaN,NaN,NaN,NaN,0,0,0,WI_3,FOODS_3_495,1.98
d_1910,2,11612,2016-04-21,Thursday,6,4,2016,NaN,NaN,NaN,NaN,0,0,0,WI_3,FOODS_3_495,1.98
d_1911,4,11612,2016-04-22,Friday,7,4,2016,NaN,NaN,NaN,NaN,0,0,0,WI_3,FOODS_3_495,1.98


# Genereting New Features

- ## Revenue
- ## Lag Day
- ## Roling Mean Revenue/Sales

In [0]:
df_final['revenue_day'] = df_final['sell_price']*df_final['qty_sales']

In [0]:
df_final['lag_1'] = df_final['qty_sales'].shift(1)
df_final['lag_2'] = df_final['qty_sales'].shift(2)
df_final['lag_3'] = df_final['qty_sales'].shift(3)
df_final['lag_4'] = df_final['qty_sales'].shift(4)
df_final['lag_5'] = df_final['qty_sales'].shift(5)
df_final['lag_6'] = df_final['qty_sales'].shift(6)
df_final['lag_7'] = df_final['qty_sales'].shift(7)

In [0]:
df_final['rolling_mean_revenue_2'] = df_final['revenue_day'].rolling(window = 2).mean()
df_final['rolling_mean_revenue_3'] = df_final['revenue_day'].rolling(window = 3).mean()
df_final['rolling_mean_revenue_4'] = df_final['revenue_day'].rolling(window = 4).mean()
df_final['rolling_mean_revenue_5'] = df_final['revenue_day'].rolling(window = 5).mean()
df_final['rolling_mean_revenue_6'] = df_final['revenue_day'].rolling(window = 6).mean()
df_final['rolling_mean_revenue_7'] = df_final['revenue_day'].rolling(window = 7).mean()

In [0]:
df_final['rolling_mean_sales_2'] = df_final['qty_sales'].rolling(window = 2).mean()
df_final['rolling_mean_sales_3'] = df_final['qty_sales'].rolling(window = 3).mean()
df_final['rolling_mean_sales_4'] = df_final['qty_sales'].rolling(window = 4).mean()
df_final['rolling_mean_sales_5'] = df_final['qty_sales'].rolling(window = 5).mean()
df_final['rolling_mean_sales_6'] = df_final['qty_sales'].rolling(window = 6).mean()
df_final['rolling_mean_sales_7'] = df_final['qty_sales'].rolling(window = 7).mean()

In [0]:
df_final = df_final[['wday','month','sell_price','revenue_day','lag_1','lag_2','lag_3','lag_4','lag_5','lag_6','lag_7','rolling_mean_revenue_2',	'rolling_mean_revenue_3',	'rolling_mean_revenue_4',	'rolling_mean_revenue_5',	'rolling_mean_revenue_6',	'rolling_mean_revenue_7',
                     'rolling_mean_sales_2',	'rolling_mean_sales_3',	'rolling_mean_sales_4',	'rolling_mean_sales_5',	'rolling_mean_sales_6',	'rolling_mean_sales_7', 'qty_sales']].fillna(0).astype(float)

In [29]:
df_final

,wday,month,sell_price,revenue_day,lag_1,lag_2,lag_3,lag_4,lag_5,lag_6,lag_7,rolling_mean_revenue_2,rolling_mean_revenue_3,rolling_mean_revenue_4,rolling_mean_revenue_5,rolling_mean_revenue_6,rolling_mean_revenue_7,rolling_mean_sales_2,rolling_mean_sales_3,rolling_mean_sales_4,rolling_mean_sales_5,rolling_mean_sales_6,rolling_mean_sales_7,qty_sales
index,,,,,,,,,,,,,,,,,,,,,,,,
d_1,1.0,1.0,1.77,1.77,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.00,0.0000,0.000,0.00,0.000000,0.0,0.000000,0.00,0.0,0.000000,0.000000,1.0
d_2,2.0,1.0,1.77,3.54,1.0,0.0,0.0,0.0,0.0,0.0,0.0,2.655,0.00,0.0000,0.000,0.00,0.000000,1.5,0.000000,0.00,0.0,0.000000,0.000000,2.0
d_3,3.0,1.0,1.77,7.08,2.0,1.0,0.0,0.0,0.0,0.0,0.0,5.310,4.13,0.0000,0.000,0.00,0.000000,3.0,2.333333,0.00,0.0,0.000000,0.000000,4.0
d_4,4.0,2.0,1.77,3.54,4.0,2.0,1.0,0.0,0.0,0.0,0.0,5.310,4.72,3.9825,0.000,0.00,0.000000,3.0,2.666667,2.25,0.0,0.000000,0.000000,2.0
d_5,5.0,2.0,1.77,5.31,2.0,4.0,2.0,1.0,0.0,0.0,0.0,4.425,5.31,4.8675,4.248,0.00,0.000000,2.5,3.000000,2.75,2.4,0.000000,0.000000,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
d_1909,5.0,4.0,1.98,1.98,2.0,5.0,4.0,1.0,3.0,4.0,4.0,2.970,5.28,5.9400,5.148,5.28,5.657143,1.5,2.666667,3.00,2.6,2.666667,2.857143,1.0
d_1910,6.0,4.0,1.98,3.96,1.0,2.0,5.0,4.0,1.0,3.0,4.0,2.970,3.30,4.9500,5.544,4.95,5.091429,1.5,1.666667,2.50,2.8,2.500000,2.571429,2.0
d_1911,7.0,4.0,1.98,7.92,2.0,1.0,2.0,5.0,4.0,1.0,3.0,5.940,4.62,4.4550,5.544,5.94,5.374286,3.0,2.333333,2.25,2.8,3.000000,2.714286,4.0


In [30]:
df_final.corr()

,wday,month,sell_price,revenue_day,lag_1,lag_2,lag_3,lag_4,lag_5,lag_6,lag_7,rolling_mean_revenue_2,rolling_mean_revenue_3,rolling_mean_revenue_4,rolling_mean_revenue_5,rolling_mean_revenue_6,rolling_mean_revenue_7,rolling_mean_sales_2,rolling_mean_sales_3,rolling_mean_sales_4,rolling_mean_sales_5,rolling_mean_sales_6,rolling_mean_sales_7,qty_sales
wday,1.000000,-0.000771,-0.000212,-0.086179,-0.063276,-0.012181,0.036394,0.092493,0.085384,-0.043217,-0.086534,-0.098609,-0.082230,-0.051899,-0.011178,0.016853,0.001431,-0.097961,-0.081787,-0.051433,-0.010929,0.016821,0.001277,-0.085983
month,-0.000771,1.000000,0.189916,-0.058242,-0.060799,-0.062983,-0.060608,-0.057803,-0.057354,-0.053459,-0.045136,-0.074820,-0.087155,-0.093741,-0.098414,-0.101950,-0.102774,-0.081466,-0.094718,-0.101845,-0.106990,-0.110872,-0.111900,-0.063398
sell_price,-0.000212,0.189916,1.000000,-0.042268,-0.067572,-0.063132,-0.058774,-0.051348,-0.046887,-0.039470,-0.036665,-0.053050,-0.057349,-0.056443,-0.053139,-0.048374,-0.038376,-0.089920,-0.099675,-0.101943,-0.101022,-0.097959,-0.088644,-0.070568
revenue_day,-0.086179,-0.058242,-0.042268,1.000000,0.152867,0.107827,0.167914,0.087377,0.107996,0.127755,0.084137,0.758868,0.643674,0.597538,0.543667,0.511491,0.487121,0.758227,0.642909,0.596536,0.542657,0.510541,0.485518,0.999454
lag_1,-0.063276,-0.060799,-0.067572,0.152867,1.000000,0.155232,0.110357,0.169216,0.088404,0.109815,0.129570,0.759208,0.667656,0.592971,0.568226,0.527058,0.500557,0.759851,0.668482,0.593921,0.568972,0.527764,0.501442,0.154693
lag_2,-0.012181,-0.062983,-0.063132,0.107827,0.155232,1.000000,0.156057,0.111039,0.169790,0.089102,0.110775,0.172251,0.644953,0.593607,0.548295,0.534524,0.502182,0.174161,0.645611,0.594602,0.549406,0.535381,0.503140,0.109199
lag_3,0.036394,-0.060608,-0.058774,0.167914,0.110357,0.156057,1.000000,0.156879,0.112038,0.170601,0.089603,0.182570,0.220886,0.600565,0.570868,0.536676,0.525769,0.183772,0.222520,0.601136,0.571766,0.537652,0.526606,0.168682
lag_4,0.092493,-0.057803,-0.051348,0.087377,0.169216,0.111039,0.156879,1.000000,0.157460,0.112716,0.171527,0.168688,0.187336,0.219260,0.547736,0.530911,0.505463,0.169498,0.188489,0.220836,0.548347,0.531781,0.506556,0.088127
lag_5,0.085384,-0.057354,-0.046887,0.107996,0.088404,0.169790,0.112038,0.157460,1.000000,0.158042,0.113924,0.129051,0.186929,0.200328,0.228428,0.517088,0.506417,0.130236,0.188027,0.201714,0.230160,0.517864,0.507587,0.109259
lag_6,-0.043217,-0.053459,-0.039470,0.127755,0.109815,0.089102,0.170601,0.112716,0.158042,1.000000,0.158972,0.155831,0.166465,0.208351,0.219394,0.243035,0.500075,0.156714,0.167388,0.209288,0.220619,0.244580,0.500921,0.128097


#Spliting Data

In [10]:
train_limit = int(round((df_food.shape[0])*0.7,0))
val_limit = int(round((df_food.shape[0])*0.85,0))

df_train = df_food[:train_limit]
df_val = df_food[train_limit:val_limit]
df_test = df_food[val_limit:]

print(df_train.shape)
print(df_val.shape)
print(df_test.shape)

(1339,)
(287,)
(287,)
